In [1]:
import cv2
import mediapipe as mp
import pydirectinput

In [2]:
mp_hand= mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [6]:
cap = cv2.VideoCapture(1)
cap.set(3, 320)  
cap.set(4, 240)

pose = ""
status = 0

frame_count = 0
skip_n = 2  # Only process every 2nd frame



with mp_hand.Hands(min_tracking_confidence=0.5,
                          min_detection_confidence=0.5 ,
                            static_image_mode=False,
                            max_num_hands= 1) as hands:
        height = 240
        width = 320
        def distance(a , b):
            a_x = int(a.x*width);
            a_y = int(a.y*height);
            b_x = int(b.x*width);
            b_y = int(b.y*height);
            return (((a_x - b_x)**2 + (a_y - b_y)**2)**0.5);
        x1 = int(0.4 * width)   # end of left zone
        x2 = int(0.6 * width)   # start of right zone (0.4 + 0.2)

        y1 = int(0.4 * height)   # end of top zone
        y2 = int(0.6 * height)   # start of bottom zone



        current_key = None
        def press_key(key):
            global current_key
            if current_key != key:
                if current_key:
                    pydirectinput.keyUp(current_key)
                pydirectinput.keyDown(key)
                current_key = key

        def release_key():
            global current_key
            if current_key:
                pydirectinput.keyUp(current_key)
                current_key = None


        while cap.isOpened():
            success, img = cap.read()
            img = cv2.flip(img, 1)   
            if not success:
                break

            height, width, _ = img.shape
   
            frame_count += 1

        # Only process pose on some frames for less lag
            if frame_count % skip_n == 0:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                result = hands.process(img_rgb)
                img = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR) 

                height , width,_ = img.shape



                try:
                    if result.multi_hand_landmarks and result.multi_handedness: # multi_hand_landmarks  are the points in the hand doesnt tell if its left or right .using multi_handedness to find whic hand
                        for hand_id, hand_landmarks in enumerate(result.multi_hand_landmarks):



                            hand_label = result.multi_handedness[hand_id].classification[0].label

                            if hand_label == "Right":
                                index_tip = hand_landmarks.landmark[mp_hand.HandLandmark.INDEX_FINGER_TIP] # can just type 8 too
                                thumb_tip = hand_landmarks.landmark[4]
                                
                                mp_drawing.draw_landmarks(
                                img,  # your current frame
                                hand_landmarks,  # landmarks to draw
                                mp_hand.HAND_CONNECTIONS,  # draw hand skeleton connections
                                mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),  # style for landmarks
                                mp_drawing.DrawingSpec(color=(0,0,255), thickness=2)  # style for connections
                            )
                                dist = distance(index_tip,thumb_tip)




                                ix = int(index_tip.x * width)
                                iy = int(index_tip.y * height)



                                if dist < 60  and status == 0:
                                    pose = "start"
                                    status = 1
                                    pydirectinput.press('space')



                                if status == 1:
                                    if ix > x2 and y1 < iy < y2:
                                        pose = "Right"
                                        press_key('d')

                                    elif ix < x1 and y1 < iy < y2:
                                        pose = "Left"
                                        press_key('a')

                                    elif iy > y2:
                                        pose = "Slide"
                                        press_key('s')

                                    elif iy < y1:
                                        pose = "Jump"
                                        press_key('w')

                                    else:
                                        pose = "Run"
                                        release_key()
                                    
                                                                

                except Exception as e:
                    print(e)
                    pass
 
            cv2.putText(img , pose , (7,25),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(255,255,0),3)     
            cv2.line(img, (x1, 0), (x1, height), (255,255,255), 2)
            cv2.line(img, (x2, 0), (x2, height), (255,255,255), 2)

            cv2.line(img, (0, y1), (width, y1), (255,255,255), 2)
            cv2.line(img, (0, y2), (width, y2), (255,255,255), 2)
                   
            
            cv2.imshow('image',img)
            if cv2.waitKey(1) == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()








